In [ ]:
#!pip install requests
#%load_ext dotenv
#%dotenv config.env

In [ ]:
import base64
import requests as rq
import datetime
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv("./config.env")
# src_file_path = inspect.getfile(lambda: None)
# print(src_file_path)
# basedir = os.path.abspath(os.path.dirname(src_file_path))
# load_dotenv(os.path.join(basedir, '.env'))

In [ ]:
client_secret = os.getenv("CLIENT_SECRET")
client_id = os.getenv("CLIENT_ID")

In [ ]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base64 string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret.")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            'grant_type' : "client_credentials"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        
        r = rq.post(token_url, data=token_data, headers=token_headers)
        #print(r.json())
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
        

In [ ]:
#base64.b64decode(client_creds_b64) #decoding

In [ ]:
spotify_client = SpotifyAPI(client_id,client_secret)
spotify_client.perform_auth()
spotify_client.access_token